In [ ]:
import sys
from config import *
print(sys.path)
path = sys.path[0]

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
full_path = f'{path}/data/2022-10-30/apartments/sale/harju/properties.csv'
data = pd.read_csv(full_path, index_col=0)
data.columns

In [ ]:
DROP_COLS = ['Date','Omandiõigus','Turvasüsteem','Krunt', 'Vannitube'
             ,'Pliit','Side','Tagatisraha','Kinnistu number'
             ,'Katus','Lisanduvad kommunaalkulud','Lisainfo','Parkimiskohti'
             ,'tasuline parkimine','Piirangud','Maakleritasu','Lisanduvad kommunaalid'
             ,'Lisaväärtused','Lisaruumid','Parkimine','tasuta parkimine','Ventilatsioon','Suve kommunaalid','Talve kommunaalid']
df = data.drop(DROP_COLS, axis=1)
df.columns

In [ ]:
COLUMN_MAPPING = {'Seisukord':'condition','Tube':'rooms','Magamistube':'bedrooms','Üldpind':'area','Korrus':'floor',
                 'Korruseid kokku':'total_floors','Hind':'price','Energiamärgis':'energy_label','Materjal':'house_material',
                 'Ehitusaasta':'year','Küte':'heating','Sanitaar':'sanitary','Lift':'lift','Rõdu':'balcony','Rõdu pind':'balcony_size','Address':'address'}
df.rename(columns=COLUMN_MAPPING, inplace=True)
df.columns

In [ ]:
df['relative_price'] = df['price']/df['area']
df = df[df['relative_price'] >500]
df.drop('relative_price', inplace=True,axis=1)
df.describe()


In [ ]:
# Dropping rows without the most important data
df = df[~df['price'].isna()]
df = df[~df['area'].isna()]
df = df[~df['floor'].isna()]
# Losing a lot of entries with removing nulls but simply using mean or dropping the column does not make sense
df = df[~df['year'].isnull()]
df = df[~df['rooms'].isnull()]


In [ ]:
df['city'] = df['District'].apply(lambda x: x.split(",")[-2])
df['district'] = df['District'].apply(lambda x: x.split(",")[-3] if len(x.split(","))>=3 else x.split(",")[-2])
df.drop('District',axis=1,inplace=True)

In [ ]:
# If not provided, assume no balcony
df['balcony_size'] = df['balcony_size'].fillna(0)
df['balcony'] = df['balcony'].fillna(0)

def check_values(col1, col2):
    """Simple function to check if boths inputs are 0."""
    response = 1 if col1!=0 or col2!=0 else 0
    return(response)

# Apply the created function for each row to see if balcony exists.    
df['flag_balcony'] = df.apply(lambda x: check_values(x.balcony_size, x.balcony), axis=1)
df['flag_lift'] = (~df['lift'].isna()).astype(int)
df.drop(['lift','balcony_size','balcony'], axis=1, inplace=True)

In [ ]:
sns.heatmap(df.isnull())

In [ ]:
df['energy_label'].unique()

In [ ]:
df['sanitary'].unique()[0:15]

In [ ]:
df['heating'].unique()[0:15]

In [ ]:
# In order to preserve as much data as possible, filling NAs. Useful for feature engineering later.
df['house_material'] = df['house_material'].fillna('Unknown')
df['heating'] = df['heating'].fillna('Unknown')
df['condition'] = df['condition'].fillna('Unknown')
df['sanitary'] = df['sanitary'].fillna('Unknown')
df['energy_label'] = df['energy_label'].fillna('energiamärgis määramata')
df = df.replace("A,A","A")

In [ ]:
#Create dummies for most important items
df['flag_shower'] = df['sanitary'].apply(lambda x: int('dušš' in x.strip().lower()))
df['flag_bath'] = df['sanitary'].apply(lambda x: int('vann' in x.strip().lower()))
df['flag_sauna'] = df['sanitary'].apply(lambda x: int('saun' in x.strip().lower()))
df['flag_generalheat'] = df['heating'].apply(lambda x: int('keskküte' in x.strip().lower()))
df['flag_floorheat'] = df['heating'].apply(lambda x: int('põrandaküte' in x.strip().lower()))
df['flag_gasheat'] = df['heating'].apply(lambda x: int('gaasiküte' in x.strip().lower()))
df['flag_electricalheat'] = df['heating'].apply(lambda x: int('õhksoojuspump' in x.strip().lower()))
df.drop('heating', axis=1, inplace=True)
df.drop('sanitary', axis=1, inplace=True)

In [ ]:
df = df[df['bedrooms']!=6]
df = df[df['rooms']<9]
df = df[df['floor']<=30.0]

In [ ]:
df.info()

In [ ]:
# Imputing with means based on most correlated features
df['total_floors'] = df['total_floors'].fillna(df.groupby('floor')['total_floors'].transform('mean').map(int))
df['bedrooms'] = df['bedrooms'].fillna(df.groupby('rooms')['bedrooms'].transform('mean').map(int))

In [ ]:
plt.figure(figsize=(16, 6))
correlations = sns.heatmap(df.corr(), annot=True)
correlations.set_title('Correlations of features', fontdict={'fontsize':12}, pad=12);

In [ ]:
df['house_material'].unique()[0:15]

In [ ]:
df['flag_brick'] = df['house_material'].apply(lambda x: int('plokkmaja' in x.strip().lower()))
df['flag_panel'] = df['house_material'].apply(lambda x: int('paneelmaja' in x.strip().lower()))
df['flag_beton'] = df['house_material'].apply(lambda x: int('betoonmaja' in x.strip().lower()))
df['flag_stone'] = df['house_material'].apply(lambda x: int('kivimaja' in x.strip().lower()))
df['flag_wood'] = df['house_material'].apply(lambda x: int('puitmaja' in x.strip().lower()))
df.drop('house_material', axis=1, inplace=True)

In [ ]:
df['condition'].unique()[0:15]

In [ ]:
df['flag_newbuilding'] = df['condition'].apply(lambda x: int('uusehitis' in x.strip().lower()))
df['flag_indevelopment'] = df['condition'].apply(lambda x: int('valmimisjärgus' in x.strip().lower()))
df['flag_renovated'] = df['condition'].apply(lambda x: int('renoveeritud' in x.strip().lower()))
df['flag_needsrenovation'] = df['condition'].apply(lambda x: int('vajab renoveerimist' in x.strip().lower()))
df.drop('condition', axis=1, inplace=True)

In [ ]:
summary = df.describe()
for col in summary:
    if col[0:4]=='flag':
        print("There are "+str(round(summary[col][1]*summary[col][0]))+" entries for "+col)

### Geocoding

In [ ]:
import requests

def get_geodata(address):
    zipcode = None
    api_key = GOOGLE_API_KEY
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    url = f"{base_url}?address={address}&key={api_key}"
    r = requests.get(url)
    if r.status_code not in range(200, 299):
        return None
    else:
        results = r.json()['results'][0]
        lat = results['geometry']['location']['lat']
        lng = results['geometry']['location']['lng']
        comps = results['address_components']
        for i in range(0,len(comps)):
            if 'postal_code' in comps[i]['types']:
                zipcode = comps[i]['short_name']
    return zipcode, lat, lng

def add_geodata(row):
    column_name = 'address'
    address_value = row[column_name]
    zipcode, lat, lng = get_geodata(address_value)
    row['zipcode'] = zipcode
    row['lat'] = lat
    row['lng'] = lng
    return row


In [ ]:
#Create zipcodes
df = df.apply(add_geodata, axis=1) # axis=1 is important to use the row itself


In [ ]:
df[['zipcode']] = df[['zipcode']].fillna(0)

In [ ]:
df.info()

In [ ]:
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

In [ ]:
df.to_csv(f'data/{today}_DataCleaned.csv')